In [ ]:
import json
import random
import os
import pandas as pd
from IPython import get_ipython
from IPython.display import display
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertTokenizerFast, BertModel
from datasets import load, load_dataset
from sklearn.model_selection import train_test_split
import evaluate
from transformers import pipeline
import glob
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import re
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, Trainer, TrainingArguments, AdamW
from datasets import Dataset

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="blairjdaniel/blairjdaniel_LLM_model")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("/Users/blairjdaniel/lighthouse/lighthouse/LLM-Sentiment-Analysis-lhl/models")
model = AutoModelForSequenceClassification.from_pretrained("/Users/blairjdaniel/lighthouse/lighthouse/LLM-Sentiment-Analysis-lhl/models")

In [ ]:
# Create a directory to store model and results
output_dir = "my_model_results" 
os.makedirs(output_dir, exist_ok=True)

# Set repo name
repo_name = "Sentiment"

# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,  # Learning rate
    weight_decay=0.01,  # Weight decay for regularization
    logging_dir='./logs',
    logging_steps=10,
    output_dir=repo_name,
    save_strategy="epoch",  # Save model at the end of each epoch
    push_to_hub=True,
)

# Define or load your model here
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()

trainer.push_to_hub()



# Save the model
trainer.save_model()  # Saves to the output_dir specified in training_args

# Save the tokenizer (optional, but recommended)
tokenizer.save_pretrained(output_dir)